Let be:
 - $\mathcal{E}$ the _electric intensity_ (in V/m)
 - $\mathcal{H}$ the _magnetic intensity_ (in A/m)
 - $\mathcal{D}$ the _electric flux density_ (in C/m²)
 - $\mathcal{J}$ the _electric current density_ (in A/m²)
 
The above quantities obey the _Maxwell equations_:
 $$\nabla \times \mathcal{H} = \frac{\partial \mathcal{D}}{\partial t} + \mathcal{J}
 $$
 When the fields are AC, that is when the time variation is harmonic, the mathematical analysis can be simplified by using complex quantities. Any AC scalar quantities can be interpreted to
 $$
 v = \sqrt{2} \Re\left( V e^{j \omega t} \right) 
 $$
 where $v$ si the _instantaneous quantity_ and $V$ the _complex quantity_ (or _phasor_). The factor $\sqrt{2}$ leads the magnitude of the complex quantity $V$ to be the effective root-mean-square (rms) value of the instantaneous quantity $v$. 
 
 The Maxwell equations in complex form are thus
 $$
 \nabla \times E = - j \omega B
 $$
 $$
 \nabla \times H = j\omega D + J
 $$
 

 Suppose we have a RF current $I$ flowing in coaxial line of diameter $D$. By applying Maxwell-Ampere law:
 $$
 \oint H \cdot dl = I
 $$
 where the _complex quantity_ $I$ is the effective root-mean-square value of the _instantaneous quantity_ $i$, following the interpretation
 $$
i = \sqrt{2} \Re\left( I e^{j \omega t} \right) 
 $$
 ie
 $$
 I = I_{rms} = I_{peak} / \sqrt{2}
 $$

Integrating around a coaxial conductor, we thus have:
$$
\pi D H_0 = I
$$
where $H_0$ is the amplitude of $H$ at the surface of the conductor ($\rho=D/2$).

A wave starting at the surface of a good conductor and propagating inward is very quickly damped to insignificant values. The field is localized in a thin layer, a phenomenon known as _skin effect_. The distance in which a wave is attenuated to $1/e$ (36.8%) of its initial value is called the _skin depth_ $\delta$, defined by:
$$
\delta =\sqrt{\frac{2}{\omega \mu \sigma}}
$$

The density of power flow into the conductor, which must also be that dissipated within the conductor, is given by:
$$
S = E \times H^* = \eta |H_0|^2 
$$
where $\eta=R + jX$ is the metal _intrinsic impedance_ and $H_0$ the amplitude of $H$ at the surface. The time-average power dissipation per unit area (in W/m²) of surface cross section the the real part of the above power flow, or:
$$
P_d = R |H_0|^2
$$
where $R$ is the _intrinsic resistance_ or the _surface impedance_ of the metal (in $\Omega$ per square):
$$
R = \frac{1}{\sigma \delta} = \sqrt{\frac{\omega \mu}{2\sigma}}
$$
see also http://www.microwaves101.com/encyclopedias/rf-sheet-resistance

Using the previous relation, the time-average power dissipation in W/m² in the conductor is thus :
$$
P_d = \frac{R}{2} \frac{|I_{peak}|^2}{(\pi D)^2} 
$$

Let be $V$ and $I$ the peak voltage and current on a coaxial transmission line under with a standing wave-ratio $S$. The maximum peak current in the line will be:

$$
I_{max}
=
\sqrt{\frac{2 P_{inc} S}{Z_0}}
$$

where $P_{inc}$ is the incident power and $Z_0$ the line characteristic impedance. 

So finally, the time-average power dissipation in W/m² in the conductor is:
$$
P_d =
 R \frac{P_{inc} S}{Z_0 (\pi D)^2}
$$ 

In [1]:
%pylab inline
from scipy.constants import epsilon_0, mu_0

Populating the interactive namespace from numpy and matplotlib


In [9]:
# Interactive widget for the IPython notebook
from ipywidgets import widgets, interact, interactive, fixed

In [3]:
# 9 inch 30 Ohm coxial line diameters [m]
line_Dint = 140e-3
line_Dout = 230e-3

# T-resonator DUT [m]
line_Dint = 128e-3
line_Dout = 219e-3

# conductor conductivity [S/m]
line_sigma_Cu = 4.4e7# 5.8e7 # inner conductor
line_sigma_Al = 3.5e7 # outer conductor

# source frequency [Hz]
line_freq = 60e6

# Line SWR
line_SWR = 2

# Input power [Watts]
line_Pinc = 0.5e6

In [10]:
# coaxial line characteristic impedance
line_Z0 = 1/(2*pi)*sqrt(mu_0/epsilon_0) * log(line_Dout/line_Dint)
print('Characteristic Impedance : Z0={} Ohm'.format(line_Z0))

def max_peak_current(Pin=1e6, SWR=1, Z0=30):
    Imax = np.sqrt(2*Pin*SWR/Z0)
    return Imax

@interact(Pin_MW=(0.1, 3, 0.1), SWR=(1,5,0.1), Z0=(10, 70, 5))
def widget_Imax(Pin_MW=line_Pinc/1e6, SWR=line_SWR, Z0=30):
    Imax = max_peak_current(Pin_MW*1e6, SWR=SWR, Z0=Z0)
    print('Maximum peak current in the line for SWR={} is I={} A'.format(line_SWR, Imax))

Maximum peak current in the line for SWR=2 is I=258.19888974716116 A


In [11]:
def ohmic_losses_heat_flux(f, I_peak, D, sigma=5.8e7):
    """
    Calculates the ohmic losses heat flux in [W/m^2] for a given current
    on a coaxial conductor
    
    Args:
        f : frequency [Hz]
        I_peak : peak current [A]
        D : conductor diameter [m]
        sigma: conductor conductivity [S/m]. Default: copper value = 5.8e7 S/m
        
    Returns:
        phi : ohmic losses heat flux [W/m^2]
    """
    # surface resistance [Ohm]
    Rs = sqrt(2.*pi*f*mu_0/(2.*sigma))
    # RF resistive losses 
    phi = Rs * abs(I_peak)**2 / 2 / (pi*D)**2
    return phi

@interact(f_MHz=(40,70,1), I_peak_A=(0,3000,1), D_mm=(50,500,10), sigma_1e7=(0, 6, 0.1))
def widget_phi(f_MHz=line_freq/1e6, I_peak_A=100, D_mm=line_Dint/1e-3, sigma_1e7=line_sigma_Cu/1e7):
    phi = ohmic_losses_heat_flux(f=f_MHz*1e6, I_peak=I_peak_A, D=D_mm*1e-3, sigma=sigma_1e7*1e7)
    print('Heat Dissipation heat flux : phi={} W/m²'.format(phi))

Heat Dissipation heat flux : phi=44875.24766190687 W/m²


In [15]:
Imax=max_peak_current(Pin=1e6, SWR=1, Z0=30)

# Ohmic loss in the inner conductor
ohmic_losses_heat_flux(line_freq, I_peak=Imax, D=line_Dint, sigma=line_sigma_Cu)

478.28660258574843

In [14]:
# Ohmic loss in the outer conductor
ohmic_losses_heat_flux(line_freq, Imax, D=line_Dout, sigma=line_sigma_Al)

183.19442017774338